# Global cutout ➜ Zarr workflow
Based on the `atlite.Cutout` core API (`atlite/cutout.py`) and the resource converters defined in `atlite/convert.py`, we can:

1. Reuse the already prepared 2019 ERA5 cutout and keep its dask-backed chunks for scalable processing.
2. Derive a global on/offshore mask from Natural Earth coastlines via `Cutout.grid`, mirroring the example approach used in the atlite notebooks.
3. Compute per-cell wind/solar capacity-factor time series with the technology-specific helpers (`Cutout.wind`, `Cutout.pv`) and merge them into an `xarray.Dataset`.
4. Persist that dataset as a consolidated Zarr store where a third variable (`is_onshore`) documents which turbine profile was applied at every location.


In [ ]:
from pathlib import Path

import atlite
import geopandas as gpd
import geodatasets
import numpy as np
import pandas as pd
import xarray as xr
from shapely.ops import unary_union

CUTOUT_PATH = Path("data/global_cutout_2019")
OUT_ZARR = Path("outputs/global_cf_2019.zarr")
CHUNKS_CF = {"time": 168, "y": 180, "x": 180}

WIND_ONSHORE_TURBINE = "Vestas_V112_3MW"
WIND_OFFSHORE_TURBINE = "NREL_ReferenceTurbine_5MW_offshore"
SOLAR_PANEL = "CSi"

# Define the global Cutout with 0.25° resolution and prepare ERA5 features required for wind+solar
cutout = atlite.Cutout(
    path=CUTOUT_PATH,
    module="era5",
    x=slice(-180, 180),
    y=slice(-90, 90),
    time="2019",
    dx=0.25,
    dy=0.25,
    overwrite=False,
    show_progress=True,
)

cutout.prepare(monthly_requests=True)
print(cutout)


/opt/homebrew/Caskroom/miniforge/base/envs/green-condor-env/lib/python3.13/site-packages/atlite/cutout.py:156: UserWarning: Arguments module, x, y, time, dx, dy, overwrite, show_progress are ignored, since cutout is already built.
  warn(


<Cutout "global_cutout_2019">
 x = -180.00 ⟷ 179.75, dx = 0.25
 y = -90.00 ⟷ 89.75, dy = 0.25
 time = 2019-01-01 ⟷ 2019-12-31, dt = h
 module = era5
 prepared_features = ['height', 'wind', 'influx', 'temperature', 'runoff']

In [ ]:
# Derive a global on/offshore mask using Natural Earth coastlines
land_shapes = gpd.read_file(geodatasets.get_path("naturalearth_land"))
land_geom = unary_union(land_shapes.geometry)

grid = cutout.grid.reset_index(drop=True)
grid["is_onshore"] = grid.geometry.centroid.within(land_geom)

mask_table = (
    grid.set_index(["y", "x"])["is_onshore"]
    .unstack("x")
    .reindex(index=cutout.coords["y"], columns=cutout.coords["x"])
    .fillna(False)
)

onshore_mask = xr.DataArray(
    mask_table.values.astype(bool),
    coords={"y": cutout.coords["y"], "x": cutout.coords["x"]},
    dims=("y", "x"),
    name="is_onshore",
).chunk({"y": CHUNKS_CF["y"], "x": CHUNKS_CF["x"]})

y_cells, x_cells = onshore_mask.shape
onshore_share = float(onshore_mask.mean().compute())
print(f"Grid shape (ny, nx): {(y_cells, x_cells)}")
print(f"Onshore share: {onshore_share:.2%}")


<xarray.Dataset> Size: 48MB
Dimensions:               (x: 9, y: 9, time: 8760)
Coordinates:
  * x                     (x) float64 72B -1.0 -0.75 -0.5 -0.25 ... 0.5 0.75 1.0
  * y                     (y) float64 72B 9.0 9.25 9.5 9.75 ... 10.5 10.75 11.0
  * time                  (time) datetime64[ns] 70kB 2019-01-01 ... 2019-12-3...
    lon                   (x) float64 72B dask.array<chunksize=(9,), meta=np.ndarray>
    lat                   (y) float64 72B dask.array<chunksize=(9,), meta=np.ndarray>
Data variables: (12/15)
    height                (y, x) float32 324B dask.array<chunksize=(9, 9), meta=np.ndarray>
    wnd100m               (time, y, x) float32 3MB dask.array<chunksize=(100, 9, 9), meta=np.ndarray>
    wnd_shear_exp         (time, y, x) float64 6MB dask.array<chunksize=(100, 9, 9), meta=np.ndarray>
    wnd_azimuth           (time, y, x) float32 3MB dask.array<chunksize=(100, 9, 9), meta=np.ndarray>
    roughness             (time, y, x) float32 3MB dask.array<chunksize=

In [ ]:
# Compute per-cell capacity factor time series using on/offshore-specific layouts
def build_unit_layout(mask: xr.DataArray | None = None) -> xr.DataArray:
    layout = cutout.data["height"].copy(deep=False)
    layout = layout.chunk({"y": CHUNKS_CF["y"], "x": CHUNKS_CF["x"]})
    layout[:] = 1.0
    if mask is not None:
        layout = layout.where(mask, 0.0)
    return layout

wind_layout_on = build_unit_layout(onshore_mask)
wind_layout_off = build_unit_layout(~onshore_mask)

cf_wind_on = cutout.wind(
    layout=wind_layout_on,
    turbine=WIND_ONSHORE_TURBINE,
    capacity_factor=True,
    per_unit=True,
)

cf_wind_off = cutout.wind(
    layout=wind_layout_off,
    turbine=WIND_OFFSHORE_TURBINE,
    capacity_factor=True,
    per_unit=True,
)

cf_wind = xr.where(onshore_mask, cf_wind_on, cf_wind_off).rename("cf_wind")

cf_solar = cutout.pv(
    layout=build_unit_layout(onshore_mask),
    panel=SOLAR_PANEL,
    orientation="latitude_optimal",
    tracking=None,
    capacity_factor=True,
    per_unit=True,
).rename("cf_solar")

cf_wind = cf_wind.chunk(CHUNKS_CF)
cf_solar = cf_solar.chunk(CHUNKS_CF)

print(cf_wind)
print(cf_solar)


Grid shape (ny, nx): (9, 9)
Onshore cells: 81
Offshore cells: 0
       y     x  on_land
0    9.0 -1.00     True
1    9.0 -0.75     True
2    9.0 -0.50     True
3    9.0 -0.25     True
4    9.0  0.00     True
..   ...   ...      ...
76  11.0  0.00     True
77  11.0  0.25     True
78  11.0  0.50     True
79  11.0  0.75     True
80  11.0  1.00     True

[81 rows x 3 columns]


/var/folders/6b/x50zyszd41z391jpwblh3ryh0000gn/T/ipykernel_78617/870556010.py:18: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  grid["on_land"] = grid.geometry.centroid.within(land_geom)
/opt/homebrew/Caskroom/miniforge/base/envs/green-condor-env/lib/python3.13/site-packages/atlite/resource.py:90: FutureWarning: 'add_cutout_windspeed' for wind turbine
power curves will default to True in atlite relase v0.2.15.
  warnings.warn(msg, FutureWarning)
/opt/homebrew/Caskroom/miniforge/base/envs/green-condor-env/lib/python3.13/site-packages/atlite/resource.py:90: FutureWarning: 'add_cutout_windspeed' for wind turbine
power curves will default to True in atlite relase v0.2.15.
  warnings.warn(msg, FutureWarning)
/opt/homebrew/Caskroom/miniforge/base/envs/green-condor-env/lib/python3.13/site-packages/atlite/resource.py:90: FutureWarning: 'add_cutout_windspe

<xarray.DataArray (time: 8760, dim_0: 1)> Size: 70kB
array([[0.44199642],
       [0.43033208],
       [0.40158301],
       ...,
       [0.22949103],
       [0.22653586],
       [0.25022666]], shape=(8760, 1))
Coordinates:
  * time     (time) datetime64[ns] 70kB 2019-01-01 ... 2019-12-31T23:00:00
  * dim_0    (dim_0) int64 8B 0
Attributes:
    units:    p.u.
<xarray.DataArray (time: 8760, dim_0: 1)> Size: 70kB
array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]], shape=(8760, 1))
Coordinates:
  * time     (time) datetime64[ns] 70kB 2019-01-01 ... 2019-12-31T23:00:00
  * dim_0    (dim_0) int64 8B 0
Attributes:
    units:    p.u.
<xarray.DataArray (time: 8760, dim_0: 1)> Size: 70kB
array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]], shape=(8760, 1))
Coordinates:
  * time     (time) datetime64[ns] 70kB 2019-01-01 ... 2019-12-31T23:00:00
  * dim_0    (dim_0) int64 8B 0
Attributes:
    units:    p.u.


In [ ]:
# Assemble dataset and persist to a consolidated Zarr store
dataset_cf = xr.Dataset(
    data_vars={
        "cf_wind": cf_wind.astype("float32"),
        "cf_solar": cf_solar.astype("float32"),
        "is_onshore": onshore_mask.astype("int8"),
    }
)

dataset_cf["cf_wind"].attrs.update(
    {
        "long_name": "Wind capacity factor",
        "per_unit_reference": f"{WIND_ONSHORE_TURBINE} onshore / {WIND_OFFSHORE_TURBINE} offshore",
    }
)
dataset_cf["cf_solar"].attrs.update(
    {
        "long_name": "Solar PV capacity factor",
        "panel": SOLAR_PANEL,
        "orientation": "latitude_optimal",
    }
)
dataset_cf["is_onshore"].attrs.update({"long_name": "1 if cell centroid on land", "dtype": "bool"})

dataset_cf = dataset_cf.chunk({"time": CHUNKS_CF["time"], "y": CHUNKS_CF["y"], "x": CHUNKS_CF["x"]})
dataset_cf["is_onshore"] = dataset_cf["is_onshore"].chunk({"y": CHUNKS_CF["y"], "x": CHUNKS_CF["x"]})

dataset_cf.attrs.update(
    {
        "source_cutout": str(CUTOUT_PATH),
        "wind_turbines": {
            "onshore": WIND_ONSHORE_TURBINE,
            "offshore": WIND_OFFSHORE_TURBINE,
        },
        "solar_panel": SOLAR_PANEL,
    }
)

OUT_ZARR.parent.mkdir(parents=True, exist_ok=True)
dataset_cf.to_zarr(OUT_ZARR, mode="w", consolidated=True)
print(f"Zarr store written to {OUT_ZARR.resolve()}")


(8760, 9, 9)


In [ ]:
# Inspect a small region directly from the Zarr store for QA
subset = {"x": slice(-1.0, 1.0), "y": slice(9.0, 11.0)}
cf_store = xr.open_zarr(OUT_ZARR, consolidated=True)
region_cf = cf_store.sel(**subset)

print(region_cf)
print(
    "Onshore cells in subset:",
    int(region_cf["is_onshore"].sum().item()),
    "| Offshore:",
    int((region_cf["is_onshore"] == 0).sum().item()),
)

pd.set_option("display.max_columns", 50)


def cf_matrix(da: xr.DataArray | None, label: str, max_hours: int = 24) -> pd.DataFrame | None:
    if da is None:
        return None

    time_dim = next((d for d in da.dims if "time" in d.lower()), da.dims[0])
    spatial_dims = [d for d in da.dims if d != time_dim]
    if not spatial_dims:
        return None

    cell_dim = "cell"
    stacked = da.stack({cell_dim: spatial_dims})

    n_cells = stacked.sizes[cell_dim]
    print(f"{label}: {n_cells} spatial cells")

    time_index = pd.Index(stacked[time_dim].values[:max_hours], name="time")
    matrix = stacked.transpose(time_dim, cell_dim).values[:max_hours, :]

    cell_index = stacked.get_index(cell_dim)
    if isinstance(cell_index, pd.MultiIndex):
        level_lookup = {name or f"level_{i}": i for i, name in enumerate(cell_index.names)}

        def find_level(tokens: tuple[str, ...]) -> int:
            for key, idx in level_lookup.items():
                lowered = key.lower()
                if any(tok in lowered for tok in tokens):
                    return idx
            return 0

        lat_level = find_level(("lat", "y"))
        lon_level = find_level(("lon", "x"))
        lat_vals = cell_index.get_level_values(lat_level).astype(float)
        lon_vals = cell_index.get_level_values(lon_level).astype(float)
        col_labels = [f"{lat:.2f}_{lon:.2f} {label}" for lat, lon in zip(lat_vals, lon_vals)]
    else:
        col_labels = [f"loc{i:03d} {label}" for i in range(n_cells)]

    df = pd.DataFrame(matrix, index=time_index, columns=col_labels)
    df[f"aggregated_{label}"] = stacked.mean(cell_dim).values[:max_hours]
    return df

matrices = [
    cf_matrix(region_cf["cf_wind"], "cf_wind"),
    cf_matrix(region_cf["cf_solar"], "cf_solar"),
]
combined = pd.concat([m for m in matrices if m is not None], axis=1)
print("Combined CF matrix shape:", combined.shape)
display(combined.head())


cf_wind_on: 1 spatial cells
cf_wind_off: 1 spatial cells
cf_solar: 1 spatial cells
Combined CF matrix shape: (24, 6)


/var/folders/6b/x50zyszd41z391jpwblh3ryh0000gn/T/ipykernel_78617/1034759064.py:37: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  lat_vals = grid.geometry.centroid.y.values[:matrix.shape[1]]
/var/folders/6b/x50zyszd41z391jpwblh3ryh0000gn/T/ipykernel_78617/1034759064.py:38: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  lon_vals = grid.geometry.centroid.x.values[:matrix.shape[1]]
/var/folders/6b/x50zyszd41z391jpwblh3ryh0000gn/T/ipykernel_78617/1034759064.py:37: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  lat_vals = grid.geometry.centroid.y.values[:matrix.shape[1]]
/var/fo

,9.00_-1.00 cf_wind_on,aggregated_cf_wind_on,9.00_-1.00 cf_wind_off,aggregated_cf_wind_off,9.00_-1.00 cf_solar,aggregated_cf_solar
time,,,,,,
2019-01-01 00:00:00,0.441996,0.441996,0.0,0.0,0.0,0.0
2019-01-01 01:00:00,0.430332,0.430332,0.0,0.0,0.0,0.0
2019-01-01 02:00:00,0.401583,0.401583,0.0,0.0,0.0,0.0
2019-01-01 03:00:00,0.367697,0.367697,0.0,0.0,0.0,0.0
2019-01-01 04:00:00,0.349065,0.349065,0.0,0.0,0.0,0.0
